In [1]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import cross_validation
import xgboost as xgb

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train_X.values, train_Y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [3]:
train=pd.read_csv("train_num_add.csv")
test=pd.read_csv("test_num_add.csv")

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (51,160,164,172,213,302,313,314,317,318,319,320,321,323,324,325,336,339,340,364,374,375,376,377,378,379,380,381,382,384,385,386,387,388,389,390,391,392,393,394,395,396,406,407) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (172,213,263,312,313,317,321,339,340,374,375,376,377,378,380,381,382,384,386,387,388,389,390,391,392,393,394,395,396) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
target=['收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']

In [5]:
#train[target].describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
count,38192.000000,38190.000000,38199.000000,38199.000000,38199.000000
mean,126.049618,76.998115,1.616563,1.406683,2.769720
std,19.275625,12.580344,1.344201,0.341184,0.852187
min,0.000000,0.000000,0.100000,0.150000,-1.200000
25%,112.000000,68.000000,0.880000,1.160000,2.180000
50%,124.000000,76.000000,1.270000,1.350000,2.690000
75%,137.000000,85.000000,1.910000,1.600000,3.260000
max,252.000000,148.000000,28.800000,4.780000,11.500000


In [6]:
train_num=train.select_dtypes(float)

In [7]:
train_num=train.select_dtypes(float)
train_X=train_num.iloc[:,5:]

In [8]:
del train_X['669001']
del train_X['31']

In [9]:
train_X=train_X.fillna(train_X.median())

In [10]:
test_new = test.loc[:, train_X.columns]  # 对test进行处理和train一样
test_new = test_new.fillna(test_new.median())  # test集缺失取中位数

In [11]:
for i in range(0,5):
    train_Y = train_num.iloc[:, i]
    train_Y = train_Y.fillna(train_Y.median())

    # model_lgb
    model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves=5,
                                  learning_rate=0.01, n_estimators=9200,
                                  max_bin=55, bagging_fraction=0.8,
                                  bagging_freq=5, feature_fraction=0.2319,
                                  feature_fraction_seed=9, bagging_seed=9,
                                  min_data_in_leaf=6, min_sum_hessian_in_leaf=11)

    model_lgb.fit(train_X, train_Y)  # 训练
    lgb_train_pred = model_lgb.predict(train_X)  # 获得训练模型
    lgb_test = model_lgb.predict(test_new)  # 得到test的预测结果
    lgb_test_pred = lgb_test.reshape(-1, 1)
    if i!=0:
        lgb_pred= np.concatenate((lgb_pred, lgb_test_pred), axis=1)
    else:
        lgb_pred=lgb_test_pred

In [12]:
for i in range(0,5):
    train_Y = train_num.iloc[:, i]
    train_Y = train_Y.fillna(train_Y.median())
    # model_xgb
    model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                                 learning_rate=0.05, max_depth=3,
                                 min_child_weight=1.7817, n_estimators=300,
                                 reg_alpha=0.4640, reg_lambda=0.8571,
                                 subsample=0.5213, silent=1,
                                 random_state=7, nthread=-1)

    model_xgb.fit(train_X, train_Y)  # 训练
    xgb_train_pred = model_xgb.predict(train_X)  # 获得训练模型
    xgb_test = model_xgb.predict(test_new)  # 得到test的预测结果
    xgb_test_pred = xgb_test.reshape(-1, 1)
    if i != 0:
        xgb_pred = np.concatenate((xgb_pred, xgb_test_pred), axis=1)
    else:
        xgb_pred = xgb_test_pred

In [13]:
lgb_pred=pd.DataFrame(lgb_pred)
print(lgb_pred.describe())
xgb_pred=pd.DataFrame(xgb_pred)
print(xgb_pred.describe())

                 0            1            2            3            4
count  9532.000000  9532.000000  9532.000000  9532.000000  9532.000000
mean    124.866325    74.707498     1.682430     1.377432     2.826589
std       9.514548     6.020823     0.741554     0.164053     0.406645
min      99.995690    59.195576     0.307312     0.879635     1.152718
25%     118.170483    70.160078     1.185623     1.260826     2.575039
50%     124.496339    74.466840     1.532368     1.362537     2.826900
75%     131.262857    78.912050     1.990138     1.475962     3.070439
max     181.701804   113.856280    13.365487     2.232538     5.388278
                 0            1            2            3            4
count  9532.000000  9532.000000  9532.000000  9532.000000  9532.000000
mean    126.431084    76.413223     1.589327     1.387598     2.896799
std       8.678664     5.759426     0.757081     0.145241     0.344377
min     104.900665    62.511486     0.543210     0.918860     1.468413
25%   

In [14]:
train[target].describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
count,38192.000000,38190.000000,38199.000000,38199.000000,38199.000000
mean,126.049618,76.998115,1.616563,1.406683,2.769720
std,19.275625,12.580344,1.344201,0.341184,0.852187
min,0.000000,0.000000,0.100000,0.150000,-1.200000
25%,112.000000,68.000000,0.880000,1.160000,2.180000
50%,124.000000,76.000000,1.270000,1.350000,2.690000
75%,137.000000,85.000000,1.910000,1.600000,3.260000
max,252.000000,148.000000,28.800000,4.780000,11.500000


In [17]:
#lgb_pred=lgb_pred_bp
#xgb_pred=xgb_pred_bp

In [18]:
#lgb_pred[1][lgb_pred[1]==0]=77.231599
#xgb_pred[1][xgb_pred[1]==0]=77.782040

In [19]:
#lgb_pred[1][lgb_pred[1]>200]=77.231599
#xgb_pred[1][xgb_pred[1]>200]=77.782040

In [20]:
pred=lgb_pred*0.7+xgb_pred*0.3;
final=pd.DataFrame(pred)
 #final=pd.DataFrame({u"收缩压":pred[:,0],u"舒张压":pred[:,1],u"血清甘油三酯":pred[:,2],u"血清高密度脂蛋白":pred[:,3],u"血清低密度脂蛋白":pred[:,4]})
file=pd.concat([test.vid,final],axis=1)
file.to_csv("meinian6.csv", index = False)

In [59]:
(lgb_pred<0).sum()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [47]:
xgb_pred=pd.DataFrame(xgb_pred)
for x in range(5):
    print(xgb_pred[x][xgb_pred[x]<0])

Series([], Name: 0, dtype: float32)
30      -575.272583
110     -272.267548
128     -125.696342
240      -19.229986
417      -36.794514
493    -1260.731079
939    -1090.840454
1104   -1324.952271
1253     -67.887833
1294     -91.284958
1304    -544.071655
1541      -7.099840
1578      -8.224051
1598   -4570.165039
1610     -51.457481
1635     -15.515310
1701    -160.929871
1867    -281.234497
1940     -87.334023
1989     -20.658056
2108    -110.023933
2150    -197.905884
2265    -167.408600
2329    -384.666779
2411     -11.122890
2463     -58.971989
2499     -73.434212
2666   -2510.638428
2841    -109.194382
2858     -20.253752
           ...     
7086     -26.539587
7087     -17.179918
7109     -22.036699
7156     -25.732227
7282   -4570.052734
7303    -569.696838
7486    -252.312012
7747     -41.764908
7910    -103.794563
7941      -4.916812
8011    -762.047607
8042     -23.226770
8081     -23.077566
8213     -21.583958
8223   -1377.360107
8281    -143.746246
8282    -304.277557
8378

In [32]:
file.describe()

,0,1,2,3,4
count,9532.000000,9532.000000,9532.000000,9532.000000,9532.000000
mean,125.335860,80.758962,1.654498,1.380483,2.847650
std,9.194030,191.506718,0.733155,0.156180,0.383901
min,101.508019,-2871.241274,0.397612,0.909420,1.248532
25%,118.863444,71.316962,1.162071,1.267494,2.612188
50%,125.061712,77.355957,1.508958,1.365274,2.852782
75%,131.622463,81.684158,1.960816,1.477328,3.078404
max,177.221681,13061.092357,12.341983,2.196862,5.384401
